<h1>Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#prepare-data" data-toc-modified-id="prepare-data-1">prepare data</a></span></li><li><span><a href="#EfficientNetB0" data-toc-modified-id="EfficientNetB0-2">EfficientNetB0</a></span></li><li><span><a href="#EfficientNetB7" data-toc-modified-id="EfficientNetB7-3">EfficientNetB7</a></span></li></ul></div>

# prepare data

In [1]:
import os
import glob
import cv2
import tensorflow as tf
import numpy as np
import json
from tqdm.notebook import tqdm
import collections

In [2]:
base_dir = os.getenv("HOME") + "/UDIGO"
model_dir = base_dir + "/models"
test_dir = base_dir + "/dataset/test"
data_dir = base_dir + "/data"

In [3]:
model_list = os.listdir(model_dir)
model_list

['15-0.60-0.83_efficient_b0_by_kec.h5',
 '17-0.68-0.84_efficient_b0_by_kec.h5',
 '11-0.54-0.84_efficient_b7_by_kyu.h5',
 '18-0.62-0.84_efficient_b0_by_kec.h5',
 '12-0.53-0.84_efficient_b7_by_kyu.h5',
 '9-0.56-0.83_efficient_b7_by_kyu.h5',
 '16-0.61-0.84_efficient_b0_by_kec.h5']

In [4]:
image_list = glob.glob(test_dir + "/*/*")
len(image_list)

10970

In [5]:
image_list[:10]

['/home/ssac21/UDIGO/dataset/test/산/349.jpg',
 '/home/ssac21/UDIGO/dataset/test/산/00002795.jpg',
 '/home/ssac21/UDIGO/dataset/test/산/178.jpg',
 '/home/ssac21/UDIGO/dataset/test/산/00003432.jpg',
 '/home/ssac21/UDIGO/dataset/test/산/67.jpg',
 '/home/ssac21/UDIGO/dataset/test/산/318.jpg',
 '/home/ssac21/UDIGO/dataset/test/산/00004403.jpg',
 '/home/ssac21/UDIGO/dataset/test/산/00002345.jpg',
 '/home/ssac21/UDIGO/dataset/test/산/108.jpg',
 '/home/ssac21/UDIGO/dataset/test/산/342.jpg']

In [6]:
with open(os.getenv("HOME") + "/UDIGO/data/place_55_label_to_id.json", "r") as f:
    label_dict = json.load(f)

# EfficientNetB0

In [7]:
model = tf.keras.models.load_model(os.path.join(model_dir, model_list[-1]))

In [8]:
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
efficientnetb0 (Functional)  (None, 1280)              4049571   
_________________________________________________________________
dense (Dense)                (None, 55)                70455     
Total params: 4,120,026
Trainable params: 4,078,003
Non-trainable params: 42,023
_________________________________________________________________


In [9]:
def inference(img_path):
    image = cv2.imread(img_path, cv2.IMREAD_COLOR)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (224, 224))
    image = image[np.newaxis, :, :, :]
    pred = model.predict(image, batch_size=1)
    return np.argmax(pred)

In [14]:
def test_model(image_list):
    label_list = []
    correct_dict = {}
    correct = 0
    n = 0

    for image in tqdm(image_list):
        try:
            pred = inference(image)
            n += 1

            label = image.split("/")[-2]
            label_list.append(label)
            if label not in correct_dict:
                correct_dict[label] = 0

            label_encoding = label_dict[label]
            if pred == label_encoding:
                correct_dict[label] += 1
                correct += 1

        except:
            print("Invalid image:", image)
            continue

    print(f"{n} data, test complete!")
    return label_list, correct_dict, correct, n

In [15]:
%%time
label_list, correct_dict, correct, n = test_model(image_list)

Invalid image: /home/ssac21/UDIGO/dataset/test/수영장/img03_01_02_04.gif
Invalid image: /home/ssac21/UDIGO/dataset/test/수상레포츠/one-footed_skiing064.gif?type=w580
Invalid image: /home/ssac21/UDIGO/dataset/test/수상레포츠/surf_board01.gif?type=w2
Invalid image: /home/ssac21/UDIGO/dataset/test/폭포/bf2698b4fe5dfcb1ac9187aa34c06699.gif
Invalid image: /home/ssac21/UDIGO/dataset/test/폭포/bf42ca3213a39ecdf97ed5141b23bc71.gif
Invalid image: /home/ssac21/UDIGO/dataset/test/워터파크/waterpark_580
Invalid image: /home/ssac21/UDIGO/dataset/test/워터파크/waterpark_001.gif?type=w1
Invalid image: /home/ssac21/UDIGO/dataset/test/볼링장/095162b2-e684-4a65-a8de-48a93307a596.gif
Invalid image: /home/ssac21/UDIGO/dataset/test/볼링장/a4b6d053-877c-48db-a5a1-dc808ea6db46.gif

10961 data, test complete!
CPU times: user 8min 9s, sys: 37.5 s, total: 8min 46s
Wall time: 6min 47s


In [16]:
count = collections.Counter(label_list)
count

Counter({'산': 200,
         '청와대': 200,
         '농구장': 200,
         '시장': 200,
         '수영장': 199,
         '미용실': 200,
         '수상레포츠': 198,
         '놀이공원': 200,
         '야구장': 198,
         '미술관': 199,
         '클럽': 200,
         '호텔': 200,
         '식물원': 199,
         '키즈카페': 200,
         '빨래방': 200,
         '대학교': 200,
         '병원': 200,
         '박물관': 196,
         '한옥마을': 200,
         '골프장': 200,
         '동물원': 200,
         '숲': 194,
         '아이스링크': 197,
         '동대문디자인플라자': 200,
         '지하철역': 200,
         '세빛섬': 200,
         '폭포': 198,
         '놀이터': 200,
         '아쿠아리움': 200,
         '절': 200,
         '다리': 200,
         'PC방': 200,
         '쇼핑몰': 200,
         '63빌딩': 200,
         '컨벤션센터': 200,
         '국회의사당': 200,
         '공원': 197,
         '롯데월드타워': 200,
         '헬스장': 200,
         '공항': 195,
         '교회': 200,
         '궁궐': 200,
         '도서관': 200,
         '워터파크': 198,
         '축구장': 200,
         '성당': 200,
         '캠핑장': 200,
     

In [17]:
correct_dict

{'산': 189,
 '청와대': 196,
 '농구장': 194,
 '시장': 187,
 '수영장': 174,
 '미용실': 178,
 '수상레포츠': 177,
 '놀이공원': 104,
 '야구장': 188,
 '미술관': 129,
 '클럽': 88,
 '호텔': 188,
 '식물원': 180,
 '키즈카페': 155,
 '빨래방': 184,
 '대학교': 152,
 '병원': 124,
 '박물관': 142,
 '한옥마을': 125,
 '골프장': 174,
 '동물원': 183,
 '숲': 190,
 '아이스링크': 162,
 '동대문디자인플라자': 187,
 '지하철역': 182,
 '세빛섬': 0,
 '폭포': 196,
 '놀이터': 154,
 '아쿠아리움': 185,
 '절': 84,
 '다리': 115,
 'PC방': 196,
 '쇼핑몰': 164,
 '63빌딩': 193,
 '컨벤션센터': 182,
 '국회의사당': 193,
 '공원': 141,
 '롯데월드타워': 156,
 '헬스장': 173,
 '공항': 136,
 '교회': 175,
 '궁궐': 189,
 '도서관': 184,
 '워터파크': 90,
 '축구장': 185,
 '성당': 149,
 '캠핑장': 174,
 '볼링장': 181,
 '패스트푸드점': 161,
 '베이커리': 180,
 '공연장': 181,
 '해수욕장': 183,
 '스키장': 188,
 '찜질방': 176,
 '남산서울타워': 186}

In [18]:
accuracy_dict = {}
for label in correct_dict:
    accuracy_dict[label] = round(correct_dict[label]/count[label], 4)

accuracy_dict

{'산': 0.945,
 '청와대': 0.98,
 '농구장': 0.97,
 '시장': 0.935,
 '수영장': 0.8744,
 '미용실': 0.89,
 '수상레포츠': 0.8939,
 '놀이공원': 0.52,
 '야구장': 0.9495,
 '미술관': 0.6482,
 '클럽': 0.44,
 '호텔': 0.94,
 '식물원': 0.9045,
 '키즈카페': 0.775,
 '빨래방': 0.92,
 '대학교': 0.76,
 '병원': 0.62,
 '박물관': 0.7245,
 '한옥마을': 0.625,
 '골프장': 0.87,
 '동물원': 0.915,
 '숲': 0.9794,
 '아이스링크': 0.8223,
 '동대문디자인플라자': 0.935,
 '지하철역': 0.91,
 '세빛섬': 0.0,
 '폭포': 0.9899,
 '놀이터': 0.77,
 '아쿠아리움': 0.925,
 '절': 0.42,
 '다리': 0.575,
 'PC방': 0.98,
 '쇼핑몰': 0.82,
 '63빌딩': 0.965,
 '컨벤션센터': 0.91,
 '국회의사당': 0.965,
 '공원': 0.7157,
 '롯데월드타워': 0.78,
 '헬스장': 0.865,
 '공항': 0.6974,
 '교회': 0.875,
 '궁궐': 0.945,
 '도서관': 0.92,
 '워터파크': 0.4545,
 '축구장': 0.925,
 '성당': 0.745,
 '캠핑장': 0.87,
 '볼링장': 0.9141,
 '패스트푸드점': 0.805,
 '베이커리': 0.9,
 '공연장': 0.9282,
 '해수욕장': 0.915,
 '스키장': 0.94,
 '찜질방': 0.88,
 '남산서울타워': 0.93}

In [19]:
accuracy = round(correct/n, 4)
print("Test Accuracy:", accuracy)

Test Accuracy: 0.8195


# EfficientNetB7

In [20]:
model = tf.keras.models.load_model(os.path.join(model_dir, model_list[-3]))

In [21]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
efficientnetb7 (Functional)  (None, 2560)              64097687  
_________________________________________________________________
dense_1 (Dense)              (None, 55)                140855    
Total params: 64,238,542
Trainable params: 63,927,815
Non-trainable params: 310,727
_________________________________________________________________


In [22]:
%%time
label_list, correct_dict, correct, n = test_model(image_list)

Invalid image: /home/ssac21/UDIGO/dataset/test/수영장/img03_01_02_04.gif
Invalid image: /home/ssac21/UDIGO/dataset/test/수상레포츠/one-footed_skiing064.gif?type=w580
Invalid image: /home/ssac21/UDIGO/dataset/test/수상레포츠/surf_board01.gif?type=w2
Invalid image: /home/ssac21/UDIGO/dataset/test/폭포/bf2698b4fe5dfcb1ac9187aa34c06699.gif
Invalid image: /home/ssac21/UDIGO/dataset/test/폭포/bf42ca3213a39ecdf97ed5141b23bc71.gif
Invalid image: /home/ssac21/UDIGO/dataset/test/워터파크/waterpark_580
Invalid image: /home/ssac21/UDIGO/dataset/test/워터파크/waterpark_001.gif?type=w1
Invalid image: /home/ssac21/UDIGO/dataset/test/볼링장/095162b2-e684-4a65-a8de-48a93307a596.gif
Invalid image: /home/ssac21/UDIGO/dataset/test/볼링장/a4b6d053-877c-48db-a5a1-dc808ea6db46.gif

10961 data, test complete!
CPU times: user 12min 12s, sys: 38.8 s, total: 12min 51s
Wall time: 11min 7s


In [23]:
correct_dict

{'산': 181,
 '청와대': 197,
 '농구장': 197,
 '시장': 190,
 '수영장': 178,
 '미용실': 161,
 '수상레포츠': 186,
 '놀이공원': 131,
 '야구장': 179,
 '미술관': 126,
 '클럽': 117,
 '호텔': 192,
 '식물원': 182,
 '키즈카페': 161,
 '빨래방': 182,
 '대학교': 162,
 '병원': 127,
 '박물관': 147,
 '한옥마을': 132,
 '골프장': 172,
 '동물원': 181,
 '숲': 177,
 '아이스링크': 174,
 '동대문디자인플라자': 192,
 '지하철역': 186,
 '세빛섬': 0,
 '폭포': 195,
 '놀이터': 147,
 '아쿠아리움': 187,
 '절': 125,
 '다리': 141,
 'PC방': 194,
 '쇼핑몰': 163,
 '63빌딩': 184,
 '컨벤션센터': 179,
 '국회의사당': 195,
 '공원': 152,
 '롯데월드타워': 162,
 '헬스장': 178,
 '공항': 160,
 '교회': 190,
 '궁궐': 183,
 '도서관': 184,
 '워터파크': 104,
 '축구장': 190,
 '성당': 165,
 '캠핑장': 177,
 '볼링장': 185,
 '패스트푸드점': 180,
 '베이커리': 190,
 '공연장': 182,
 '해수욕장': 180,
 '스키장': 193,
 '찜질방': 184,
 '남산서울타워': 190}

In [24]:
accuracy_dict = {}
for label in correct_dict:
    accuracy_dict[label] = round(correct_dict[label]/count[label], 4)

accuracy_dict

{'산': 0.905,
 '청와대': 0.985,
 '농구장': 0.985,
 '시장': 0.95,
 '수영장': 0.8945,
 '미용실': 0.805,
 '수상레포츠': 0.9394,
 '놀이공원': 0.655,
 '야구장': 0.904,
 '미술관': 0.6332,
 '클럽': 0.585,
 '호텔': 0.96,
 '식물원': 0.9146,
 '키즈카페': 0.805,
 '빨래방': 0.91,
 '대학교': 0.81,
 '병원': 0.635,
 '박물관': 0.75,
 '한옥마을': 0.66,
 '골프장': 0.86,
 '동물원': 0.905,
 '숲': 0.9124,
 '아이스링크': 0.8832,
 '동대문디자인플라자': 0.96,
 '지하철역': 0.93,
 '세빛섬': 0.0,
 '폭포': 0.9848,
 '놀이터': 0.735,
 '아쿠아리움': 0.935,
 '절': 0.625,
 '다리': 0.705,
 'PC방': 0.97,
 '쇼핑몰': 0.815,
 '63빌딩': 0.92,
 '컨벤션센터': 0.895,
 '국회의사당': 0.975,
 '공원': 0.7716,
 '롯데월드타워': 0.81,
 '헬스장': 0.89,
 '공항': 0.8205,
 '교회': 0.95,
 '궁궐': 0.915,
 '도서관': 0.92,
 '워터파크': 0.5253,
 '축구장': 0.95,
 '성당': 0.825,
 '캠핑장': 0.885,
 '볼링장': 0.9343,
 '패스트푸드점': 0.9,
 '베이커리': 0.95,
 '공연장': 0.9333,
 '해수욕장': 0.9,
 '스키장': 0.965,
 '찜질방': 0.92,
 '남산서울타워': 0.95}

In [25]:
accuracy = round(correct/n, 4)
print("Test Accuracy:", accuracy)

Test Accuracy: 0.8438
